<a href="https://colab.research.google.com/github/Manshrishevde/zomato-recommendation-system/blob/main/zomato_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# ZOMATO RECOMMENDATION SYSTEM

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
#reading the dataset
zomato_real=pd.read_csv("/content/zomato.csv")

In [55]:


zomato_real.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [ ]:
zomato_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [ ]:
zomato_real.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


In [ ]:
zomato_real.shape

(51717, 17)

In [ ]:
zomato_real.ndim

2

In [ ]:
# Remove the nan values from the dataset
zomato_real.isnull().sum()

,0
url,0
address,0
name,0
online_order,0
book_table,0
rate,7775
votes,0
phone,1208
location,21
rest_type,227


In [ ]:
# ?chhanging collumn names

zomato_real = zomato_real.rename(columns = {'approx_cost(for two people)' : 'cost', 'listed_in(type)' : 'type', 'listed_in(city)': 'city'})

In [ ]:
zomato_real

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [ ]:
zomato_real.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines', 'cost',
       'reviews_list', 'menu_item', 'type', 'city'],
      dtype='object')

In [ ]:
zomato_real.duplicated().sum()

0

In [ ]:
zomatoo=zomato_real.drop(['url','dish_liked','phone'],axis=1)

In [ ]:
zomatoo

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,Whitefield,Bar,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,Whitefield,Bar,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,Whitefield,Bar,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,"ITPL Main Road, Whitefield",Bar,Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [ ]:
zomatoo.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [ ]:
zomatoo.duplicated().sum()

43

In [ ]:
zomatoo.drop_duplicates(inplace=True)

In [ ]:
zomatoo.duplicated().sum()

0

In [ ]:
zomatoo.isnull().sum()

,0
address,0
name,0
online_order,0
book_table,0
rate,7767
votes,0
location,21
rest_type,227
cuisines,45
cost,345


In [ ]:
zomatoo.dropna(how='any',inplace=True)

In [ ]:
zomatoo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       43499 non-null  object
 1   name          43499 non-null  object
 2   online_order  43499 non-null  object
 3   book_table    43499 non-null  object
 4   rate          43499 non-null  object
 5   votes         43499 non-null  int64 
 6   location      43499 non-null  object
 7   rest_type     43499 non-null  object
 8   cuisines      43499 non-null  object
 9   cost          43499 non-null  object
 10  reviews_list  43499 non-null  object
 11  menu_item     43499 non-null  object
 12  type          43499 non-null  object
 13  city          43499 non-null  object
dtypes: int64(1), object(13)
memory usage: 5.0+ MB


In [ ]:
zomatoo = zomatoo.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'})

In [ ]:
zomatoo.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [ ]:
zomatoo['cost'] = zomatoo['cost'].astype(str) #Changing the cost to string
zomatoo['cost'] = zomatoo['cost'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from cost
zomatoo['cost'] = zomatoo['cost'].astype(float) # Changing the cost to Float

In [ ]:
zomatoo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       43499 non-null  object 
 1   name          43499 non-null  object 
 2   online_order  43499 non-null  object 
 3   book_table    43499 non-null  object 
 4   rate          43499 non-null  object 
 5   votes         43499 non-null  int64  
 6   location      43499 non-null  object 
 7   rest_type     43499 non-null  object 
 8   cuisines      43499 non-null  object 
 9   cost          43499 non-null  float64
 10  reviews_list  43499 non-null  object 
 11  menu_item     43499 non-null  object 
 12  type          43499 non-null  object 
 13  city          43499 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 5.0+ MB


In [ ]:
zomatoo['cost']

,cost
0,800.0
1,800.0
2,800.0
3,300.0
4,600.0
...,...
51709,800.0
51711,800.0
51712,1500.0
51715,2500.0


In [ ]:
zomatoo['cost'].sample(20)

,cost
32470,500.0
4100,300.0
13263,1000.0
1469,400.0
40380,350.0
27973,500.0
37803,1100.0
48324,400.0
10279,500.0
10929,750.0


In [ ]:
zomatoo['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [ ]:
# Remove rows where 'rate' is 'NEW' or '-'
zomatoo = zomatoo.loc[~zomatoo['rate'].isin(['NEW', '-'])].reset_index(drop=True)

# Define a function to remove '/5' and apply it
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x
zomatoo['rate'] = zomatoo['rate'].apply(remove_slash).str.strip().astype(float)


In [ ]:
zomatoo['rate'].head()

,rate
0,4.1
1,4.1
2,3.8
3,3.7
4,3.8


In [ ]:
zomatoo['rate'].sample(20)

,rate
7338,3.5
41095,2.7
11806,3.9
14250,3.2
38703,3.1
20585,4.4
11769,3.9
14730,4.2
12888,4.2
24256,3.7


In [ ]:
# Adjust the column names
zomatoo.name = zomatoo.name.apply(lambda x:x.title())
zomatoo.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomatoo.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [ ]:
zomatoo.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [ ]:
zomatoo['online_order'].sample(5)

,online_order
13320,False
13660,True
10905,True
26219,True
36008,True


In [ ]:
zomatoo['book_table'].sample(5)

,book_table
16164,False
17055,False
33686,False
24621,False
31296,True


In [ ]:
zomatoo.cost.unique()

array([ 800.,  300.,  600.,  700.,  550.,  500.,  450.,  650.,  400.,
        900.,  200.,  750.,  150.,  850.,  100., 1200.,  350.,  250.,
        950., 1000., 1500., 1300.,  199., 1100., 1600.,  230.,  130.,
       1700., 1350., 2200., 1400., 2000., 1800., 1900.,  180.,  330.,
       2500., 2100., 3000., 2800., 3400.,   50.,   40., 1250., 3500.,
       4000., 2400., 2600., 1450.,   70., 3200.,  240., 6000., 1050.,
       2300., 4100.,  120., 5000., 3700., 1650., 2700., 4500.,   80.])

In [ ]:
zomatoo['city'].unique()

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [ ]:
zomatoo.isnull().sum()

,0
address,0
name,0
online_order,0
book_table,0
rate,0
votes,0
location,0
rest_type,0
cuisines,0
cost,0


In [ ]:
zomatoo.duplicated().sum()

0

In [ ]:
## Computing Mean Rating
restaurants = list(zomatoo['name'].unique())
zomatoo['Mean Rating'] = 0

In [ ]:
for i in range(len(restaurants)):
    zomatoo['Mean Rating'][zomatoo['name'] == restaurants[i]] = zomatoo['rate'][zomatoo['name'] == restaurants[i]].mean()

In [ ]:
zomatoo.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,0
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,0
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,0
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,0
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,0


In [56]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (1,5))

zomatoo[['Mean Rating']] = scaler.fit_transform(zomatoo[['Mean Rating']]).round(2)

In [57]:
zomatoo.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
18917,"406, 5Th Main, 2nd Block, HRBR Layout, Banaswa...",The Good Bowl,True,False,3.5,15,Kalyan Nagar,"Takeaway, Delivery","Italian, North Indian, Biryani",500.0,"[('Rated 4.0', ""RATED\n The name surely says ...",[],Delivery,Kalyan Nagar,1.0
30690,"57/1, 1st Floor, Jayalaxmi Chambers, Next to O...",Meghana Foods,True,False,4.3,2151,Residency Road,Casual Dining,"Biryani, Andhra, North Indian, Seafood",600.0,"[('Rated 4.0', 'RATED\n Went with colleagues ...",[],Dine-out,Lavelle Road,1.0
30144,"1A, Church Street, Bangalore",Wolf'Ish,True,False,4.1,481,Church Street,Cafe,"Cafe, American, Italian, Pizza, Burger",700.0,"[('Rated 4.0', ""RATED\n Visited this place wi...",[],Delivery,Lavelle Road,1.0
33328,"Kaverappa Layout, Kadubeesanahalli, Marathahal...",The Zaika Of Banaras,True,False,3.3,26,Marathahalli,Quick Bites,"North Indian, Street Food",350.0,"[('Rated 3.0', 'RATED\n What a disaster for m...",[],Dine-out,Marathahalli,1.0
18970,"133, Lingarajapuram, Below Fly Over, Near Kamm...",Hotel Zam Zam,True,False,3.2,30,Hennur,Quick Bites,"North Indian, Chinese",300.0,"[('Rated 1.0', ""RATED\n As the saying goes 'y...",[],Delivery,Kalyan Nagar,1.0


In [58]:
## Text Preprocessing

# Lower casing
# Removal of Punctuations
# Removal of Stopwords
# Removal of URLs
# Spelling correction

In [59]:
zomatoo["reviews_list"] = zomatoo["reviews_list"].str.lower()
zomatoo["cuisines"] = zomatoo["cuisines"].str.lower()

In [60]:
zomatoo[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
12223,"[('rated 4.0', 'rated\n good place to have th...","fast food, chinese"
4874,"[('rated 5.0', 'rated\n food is always excell...","european, italian, desserts, salad, juices, steak"
9109,"[('rated 4.0', ""rated\n a 4.5.\nvery good pla...","south indian, andhra"
10372,"[('rated 4.0', 'rated\n located at a prime pl...","beverages, desserts"
19733,"[('rated 3.0', 'rated\n this outlet is not as...","fast food, burger"


In [61]:
import string

In [62]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomatoo["reviews_list"] = zomatoo["reviews_list"].apply(lambda text: remove_punctuation(text))

In [63]:
zomatoo[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
39138,rated 30 ratedn seeing positive reviewsi orde...,andhra
1793,rated 50 ratedn kalmi kabab chicken korma chi...,"north indian, chinese, biryani"
36175,rated 50 ratedn purely home made food awsome ...,north indian
28026,rated 50 ratedn punjabi cuisine is one of my ...,north indian
6310,rated 10 ratedn i ordered khara and sweet bat...,south indian


In [65]:
# stopwords removal
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomatoo["reviews_list"] = zomatoo["reviews_list"].apply(lambda text: remove_stopwords(text))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [66]:
!pip install nltk
import nltk
nltk.download('stopwords')

# stopwords removal
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomatoo["reviews_list"] = zomatoo["reviews_list"].apply(lambda text: remove_stopwords(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [67]:
zomatoo["reviews_list"][5]

'rated 30 ratedn food 35nambience 35nservice 35nnhad family lunch place serves buffets ordered soups babycorn starters butter naan kadai panner veg kohlapuri pease pulav food good service slow make wait 20 mins get order place apt family hangoutnncheers'

In [68]:
# Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [69]:
zomatoo["reviews_list"] = zomatoo["reviews_list"].apply(lambda text: remove_urls(text))

In [70]:
zomatoo["reviews_list"][1]

'rated 40 ratedn dinner family turned good choose suitable ages people try place liked starters service good prices affordable recommend restaurant early dinner place little noisy rated 30 ratedn ambience really nice staff courteous price pretty high quantity overall experience fine quality food nice nothing extraordinary also buffetonly veg rated 30 ratedn felt good little expensive quantity serve terms taste decent nothing much talk ambience regular casual dining restaurant take family dinner lunch improve quantity may reduce price bit may improve presentation food might manage get repeat customers rated 40 ratedn looking quite place spend time family well wanted try new place since banashankari thought trying place place good rating part zomato gold decided try place delite see friendly staff food ordered tasty wellnnfood 45nambience 35nfriendly staff 45npocket friendly 45nnwill definitely visit rated 40 ratedn nice place dine good ambiance food good serving time also goodneat restr

In [73]:
restaurant_names = list(zomatoo['name'].unique())

In [74]:
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [75]:
def get_top_words(column, top_nu_of_words, nu_of_word):

    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')

    bag_of_words = vec.fit_transform(column)

    sum_words = bag_of_words.sum(axis=0)

    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return words_freq[:top_nu_of_words]

In [76]:
zomatoo.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
25327,"17th Main Road, JNC Road, Koramangala 5th Bloc...",The Reservoire,False,True,4.6,2590,Koramangala 5th Block,"Bar, Casual Dining","continental, north indian, chinese, american",1300.0,rated 40 ratedn went sunday brunch food great ...,[],Pubs and bars,Koramangala 5th Block,1.0
39936,"3/2, 1st Floor,Near Ramdev Medical, Bellandur ...",Ammi'S Biryani,False,False,2.8,43,Bellandur,Quick Bites,"biryani, north indian",500.0,rated 10 ratedn never worest quality rated 35 ...,[],Dine-out,Sarjapur Road,1.0
27552,"562, 8th Main, Koramangala 4th Block, Bangalore",Onesta,True,True,4.4,9084,Koramangala 4th Block,"Casual Dining, Cafe","pizza, cafe, italian",600.0,rated 40 ratedn onesta sunday evening even tho...,[],Cafes,Koramangala 7th Block,1.0
23664,"59, Ground Floor, Opposite Jyoti Niwas College...",Chai Galli,True,False,4.4,480,Koramangala 5th Block,Cafe,"cafe, tea, street food",450.0,rated 40 ratedn went place friends ordered mas...,"['Bhel Puri', 'Chai Galli Ki Vada Pav', 'Poha'...",Delivery,Koramangala 5th Block,1.0
13755,"145, Ground Floor, Below Swathi Spice n Ice, C...",Safe House,False,True,4.1,1372,HSR,"Bar, Casual Dining","continental, north indian, chinese, finger food",900.0,rated 30 ratedn pictures deceiving research sw...,[],Drinks & nightlife,HSR,1.0


In [77]:
zomatoo=zomatoo.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [78]:
zomatoo.head()

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,Banashankari,"north indian, mughlai, chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,1.0
1,Spice Elephant,True,False,4.1,Banashankari,"chinese, north indian, thai",800.0,rated 40 ratedn dinner family turned good choo...,Banashankari,1.0
2,San Churro Cafe,True,False,3.8,Banashankari,"cafe, mexican, italian",800.0,rated 30 ratedn ambience good enough pocket fr...,Banashankari,1.0
3,Addhuri Udupi Bhojana,False,False,3.7,Banashankari,"south indian, north indian",300.0,rated 40 ratedn great food proper karnataka st...,Banashankari,1.0
4,Grand Village,False,False,3.8,Basavanagudi,"north indian, rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,1.0


In [79]:
# Randomly sample 60% of your dataframe
df_percent = zomatoo.sample(frac=0.5)

In [80]:
df_percent.shape

(20618, 10)

In [81]:
df_percent.set_index('name', inplace=True)

In [82]:
indices = pd.Series(df_percent.index)

In [84]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english') # Changed min_df to 1
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

In [85]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [86]:
def recommend(name, cosine_similarities = cosine_similarities):

    # Create a list to put top 10 restaurants
    recommend_restaurant = []

    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]

    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)

    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))

    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))

    return df_new

In [87]:
df_percent[df_percent.index == 'Pai Vihar'].head()

,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,,,
Pai Vihar,False,False,3.2,City Market,"south indian, street food, chinese, fast food",400.0,rated 20 ratedn food dry bland dont understand...,Church Street,1.0
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,Malleshwaram,1.0
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate herenneven though tast...,Frazer Town,1.0
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate herenneven though tast...,Frazer Town,1.0
Pai Vihar,False,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,Residency Road,1.0


In [88]:
recommend('Pai Vihar')

AttributeError: 'DataFrame' object has no attribute 'append'

In [90]:
def recommend(name, cosine_similarities = cosine_similarities):

    # Create a list to put top 10 restaurants
    recommend_restaurant = []

    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]

    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)

    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)

    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])

    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

    # Create the top 30 similar restaurants with some of their columns
    # Use pd.concat instead of append
    df_list = []  # Create a list to store DataFrames
    for each in recommend_restaurant:
        df_list.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))

    df_new = pd.concat(df_list, ignore_index=True) # Concatenate all DataFrames in the list

    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))

    return df_new

In [91]:
recommend('Jalsa')

TOP 9 RESTAURANTS LIKE Jalsa WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
1,"north indian, mughlai",1.0,1500.0
5,"italian, mediterranean",1.0,2500.0
6,"biryani, north indian, chinese",1.0,600.0
14,"north indian, continental, italian",1.0,900.0
20,north indian,1.0,500.0
22,"biryani, chinese, andhra",1.0,500.0
27,"continental, north indian, bbq, steak",1.0,1400.0
28,"continental, italian, north indian",1.0,1400.0
29,"north indian, chinese",1.0,1200.0


In [92]:
recommend('Timepass Dinner')

TOP 3 RESTAURANTS LIKE Timepass Dinner WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
27,north indian,1.0,500.0
28,"chinese, north indian, continental",1.0,750.0
30,"asian, chinese, continental, north indian, sou...",1.0,1300.0
